In [20]:
import pandas as pd
import numpy as np

from pandas.plotting import scatter_matrix
from seaborn import scatterplot, heatmap

from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.preprocessing import OneHotEncoder

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

from sklearn.metrics import mean_absolute_error

from joblib import dump

import os
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  base_dir = "./drive/My Drive/Colab Notebooks/" # You may need to change this, depending on where your notebooks are on Google Drive
else:
  base_dir = "." 

In [21]:
df = pd.read_csv(os.path.join(base_dir, "../datasets/dataset_concrete.csv"))

#See our dataset size
df.describe()

,cement,slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [22]:
# Split off the test set: 20% of the dataset.
dev_df, test_df = train_test_split(df, train_size=0.8, random_state=2)
# Extract the features but leave as a DataFrame
features = ["cement", "slag", "fly_ash", "water", "superplasticizer", "coarse_aggregate", "fine_aggregate", "age"]
dev_X = dev_df[features]
test_X = test_df[features]

# Target values, converted to a 1D numpy array
dev_y = dev_df["strength"].values
test_y = test_df["strength"].values

In [23]:
# Create the object that shuffles and splits the dev data
# Why 0.75? Because 0.75 of 80% of the data is 60% of the original dataset.
ss = ShuffleSplit(n_splits=1, train_size=0.75, random_state=2)
# Create a preprocessor
preprocessor = ColumnTransformer([
        ("scaler", StandardScaler(), features)], 
        remainder="drop")


NameError: name 'ShuffleSplit' is not defined

In [8]:
#Linear Model
linear_model = Pipeline([
    ("preprocessor", preprocessor),
    ("predictor", LinearRegression())])

NameError: name 'preprocessor' is not defined

In [9]:
# Create a pipeline that combines the preprocessor with 1NN
knn_model = Pipeline([
    ("preprocessor", preprocessor),
    ("predictor", KNeighborsRegressor(n_neighbors=1))])

knn_param = {"predictor__n_neighbors" : [1,2,3,4,5]}

# Error estimation for k=1
#np.mean(cross_val_score(knn_model, dev_X, dev_y, scoring="neg_mean_absolute_error", cv=10))

NameError: name 'preprocessor' is not defined

In [10]:
knn_gs = GridSearchCV(knn_model, knn_param, scoring="neg_mean_absolute_error", cv=10, refit=True)
knn_gs.fit(dev_X, dev_y)
knn_gs.best_params_, knn_gs.best_score_

NameError: name 'knn_model' is not defined

In [11]:


linear_model.fit(dev_X, dev_y)

# Error estimation on the test set.
mean_absolute_error(test_y, linear_model.predict(test_X))

NameError: name 'linear_model' is not defined

In [12]:
mean_absolute_error(test_y, knn_gs.predict(test_X))

NameError: name 'knn_gs' is not defined

In [13]:
knn_gs.fit(df[features], df["strength"].values)

NameError: name 'knn_gs' is not defined

<h2> Lab 4</h2>

In [14]:
dev_df

,cement,slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,strength
128,401.8,94.7,0.0,147.4,11.4,946.8,852.1,28,68.50
365,214.9,53.8,121.9,155.6,9.6,1014.3,780.6,14,38.60
480,446.0,24.0,79.0,162.0,11.6,967.0,712.0,7,39.30
814,310.0,0.0,0.0,192.0,0.0,970.0,850.0,360,38.11
169,425.0,106.3,0.0,153.5,16.5,852.1,887.1,91,65.20
...,...,...,...,...,...,...,...,...,...
360,218.2,54.6,123.8,140.8,11.9,1075.7,792.7,14,35.96
466,190.3,0.0,125.2,166.6,9.9,1079.0,798.9,100,33.56
299,290.4,0.0,96.2,168.1,9.4,961.2,865.0,3,22.50
493,387.0,20.0,94.0,157.0,11.6,938.0,845.0,7,41.67


In [15]:
#Copy data frame
copy_df = dev_df.copy()
copy_df["wcr"] = copy_df["water"] / copy_df["cement"]
#copy_df["aggregate_total"] = copy_df["coarse_aggregate"] + copy_df["fine_aggregate"]


In [16]:
class InsertWCR(BaseEstimator, TransformerMixin):

    def __init__(self, insert=True):
        self.insert = insert
        
    def fit(self, X, y=None):
        return X
    
    def transform(self, X, y=None):
        if self.insert:
            X["wcr"] = X["water"] / (X["cement"])
            
            # If the new feature is intended to replace the existing ones, 
            # you could drop the existing ones here
            # X.drop(["flarea", "bthrms", "bdrms"], axis=1)
    
            #X = X.replace( [ np.inf, -np.inf ], np.nan )
        return X
        

In [17]:
preprocessor = ColumnTransformer([
    ("features", Pipeline([("wcr", InsertWCR()),
    ("scaler", StandardScaler())]),
features)],
remainder="drop")

dev_df

,cement,slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,strength
128,401.8,94.7,0.0,147.4,11.4,946.8,852.1,28,68.50
365,214.9,53.8,121.9,155.6,9.6,1014.3,780.6,14,38.60
480,446.0,24.0,79.0,162.0,11.6,967.0,712.0,7,39.30
814,310.0,0.0,0.0,192.0,0.0,970.0,850.0,360,38.11
169,425.0,106.3,0.0,153.5,16.5,852.1,887.1,91,65.20
...,...,...,...,...,...,...,...,...,...
360,218.2,54.6,123.8,140.8,11.9,1075.7,792.7,14,35.96
466,190.3,0.0,125.2,166.6,9.9,1079.0,798.9,100,33.56
299,290.4,0.0,96.2,168.1,9.4,961.2,865.0,3,22.50
493,387.0,20.0,94.0,157.0,11.6,938.0,845.0,7,41.67


In [18]:
# Extract the features but leave as a DataFrame
dev_X = dev_df[features]
test_X = test_df[features]

# Target values, converted to a 1D numpy array
dev_y = dev_df["strength"].values
test_y = test_df["strength"].values

In [19]:
# Create a pipeline that combines the preprocessor with kNN
ols = Pipeline([
    ("preprocessor", preprocessor),
    ("predictor", LinearRegression())])

# Create a dictionary of hyperparameters for kNN
ols_param_grid = {"preprocessor__features__wcr__insert": [True, False]}

# Create the grid search object which will find the best hyperparameter values based on validation error
ols_gs = GridSearchCV(ols, ols_param_grid, scoring="neg_mean_absolute_error", cv=10, refit=True)

# Run grid search by calling fit. It will also re-train on train+validation using the best parameters.
ols_gs.fit(dev_X, dev_y)

# Let's see how well we did
ols_gs.best_params_, knn_gs.best_score_

NameError: name 'LinearRegression' is not defined